In [60]:
include("schedule_lib.jl")

import_excel (generic function with 1 method)

In [61]:
# to be able to add new methods to the existing ones in Julia Base
import Base.getindex, Base.get, Base.haskey, Base.in, Base.copy, Base.deepcopy

abstract type AbstractProblem end;
abstract type AbstractCSP <: AbstractProblem end;

# ___________________________________________________________________ #
# Everything related to dictionaries for domain and neighbor tracking #
# ___________________________________________________________________ #
struct ConstantFunctionDict{V}
    value::V
    function ConstantFunctionDict{V}(val::V) where V
           return new(val);
    end
end

ConstantFunctionDict(val) = ConstantFunctionDict{typeof(val)}(val);

copy(cfd::ConstantFunctionDict) = ConstantFunctionDict{typeof(cfd.value)}(cfd.value);

deepcopy(cfd::ConstantFunctionDict) = ConstantFunctionDict{typeof(cfd.value)}(deepcopy(cfd.value));

mutable struct CSPDict
    dict::Union{Nothing, Dict, ConstantFunctionDict}
    
    function CSPDict(dictionary::Union{Nothing, Dict, ConstantFunctionDict})
        return new(dictionary);
    end
end

function getindex(dict::CSPDict, key)
    if (typeof(dict.dict) <: ConstantFunctionDict)
        return dict.dict.value;
    else
        return getindex(dict.dict, key);
    end
end

function getkey(dict::CSPDict, key, default)
    if (typeof(dict.dict) <: ConstantFunctionDict)
        return dict.dict.value;
    else
        return getkey(dict.dict, key, default);
    end
end

function get(dict::CSPDict, key, default)
    if (typeof(dict.dict) <: ConstantFunctionDict)
        return dict.dict.value;
    else
        return get(dict.dict, key, default);
    end
end

function haskey(dict::CSPDict, key)
    if (typeof(dict.dict) <: ConstantFunctionDict)
        return true;
    else
        return haskey(dict.dict, key);
    end
end

function in(pair::Pair, dict::CSPDict)
    if (typeof(dict.dict) <: ConstantFunctionDict)
        if (getindex(pair, 2) == dict.dict.value)
            return true;
        else
            return false;
        end
    else
        return in(pair, dict.dict);
    end
end

# ___________________________________________________________________ #
# Everything related to assigning variables #
# ___________________________________________________________________ #

"""
    assign(problem, key, val, assignment)
    Overwrite (if an value exists already) assignment[key] with 'val'.
"""
function assign(problem::T, key, val, assignment::Dict) where {T <: AbstractCSP}
    assignment[key] = val;
    problem.nassigns = problem.nassigns + 1;
    nothing;
end

"""
    unassign(problem, key, val, assignment)
    Delete the existing (key, val) pair from 'assignment'.
"""
function unassign(problem::T, key, assignment::Dict) where {T <: AbstractCSP}
    if (haskey(assignment, key))
        delete!(assignment, key);
    end
    nothing;
end

function nconflicts(problem::T, key, val, assignment::Dict) where {T <: AbstractCSP}
    return count((
        function(second_key)
            return (haskey(assignment, second_key) && !(problem.constraints(key, val, second_key, assignment[second_key])));
        end),
    problem.neighbors[key]);
end


function support_pruning(problem::T) where {T <: AbstractCSP}
    if (problem.current_domains === nothing)
        problem.current_domains = Dict(collect(Pair(key, collect(problem.domains[key])) for key in problem.vars));
    end
    nothing;
end

function suppose(problem::T, key, val) where {T <: AbstractCSP}
    support_pruning(problem);
    local removals::AbstractVector = collect(Pair(key, a) for a in problem.current_domains[key]
    if (a != val));
        problem.current_domains[key] = [val];
    return removals;
end


function parse_neighbors(neighbors::String; vars::AbstractVector=[])
    local new_dict = Dict();
    for var in vars
        new_dict[var] = [];
    end
    local specs::AbstractVector = collect(map(String, split(spec, [':'])) for spec in split(neighbors, [';']));
    for (A, A_n) in specs
        A = String(strip(A));
        if (!haskey(new_dict, A))
            new_dict[A] = [];
        end
        for B in map(String, split(A_n))
            push!(new_dict[A], B);
            if (!haskey(new_dict, B))
                new_dict[B] = [];
            end
            push!(new_dict[B], A);
        end
    end
    return new_dict;
end

parse_neighbors (generic function with 1 method)

In [62]:
function backtracking_search(problem::T; select_unassigned_variable::Function=first_unassigned_variable,
        order_domain_values::Function=unordered_domain_values,
        inference::Function=no_inference) where {T <: AbstractCSP}
    local result = backtrack(problem, Dict(),select_unassigned_variable=select_unassigned_variable,order_domain_values=order_domain_values,inference=inference);
    if (!(typeof(result) <: Nothing || goal_test(problem, result)))
        error("BacktrackingSearchError: Unexpected result!")
    end
    return result;
end

function backtrack(problem::T, assignment::Dict; select_unassigned_variable::Function=first_unassigned_variable, order_domain_values::Function=unordered_domain_values, inference::Function=no_inference) where {T <: AbstractCSP}
    if (length(assignment) == length(problem.vars))
        return assignment;
    end
    local var = select_unassigned_variable(problem, assignment);
    for value in order_domain_values(problem, var, assignment)
        if (nconflicts(problem, var, value, assignment) == 0)
            assign(problem, var, value, assignment);
            removals = suppose(problem, var, value);
            if (inference(problem, var, value, assignment, removals))
                result = backtrack(problem, assignment, select_unassigned_variable=select_unassigned_variable,order_domain_values=order_domain_values,inference=inference);
                if (!(typeof(result) <: Nothing))
                    return result;
                end
            end
            restore(problem, removals);
        end
    end
    unassign(problem, var, assignment);
    return nothing;
end


function restore(problem::T, removals::AbstractVector) where {T <: AbstractCSP}
    for (key, val) in removals
        push!(problem.current_domains[key], val);
    end
    nothing;
end

function goal_test(problem::T, state::Tuple) where {T <: AbstractCSP}
    let
    local assignment = Dict(state);
    return (length(assignment) == length(problem.vars) && all((function(key) return nconflicts(problem, key, assignment[key], assignment) == 0; end),problem.vars));
    end
end

function goal_test(problem::T, state::Dict) where {T <: AbstractCSP}
    let
    local assignment = deepcopy(state);
    return (length(assignment) == length(problem.vars) && all((function(key) return nconflicts(problem, key, assignment[key], assignment) == 0; end),problem.vars));
    end
end


function first_unassigned_variable(problem::T, assignment::Dict) where {T <: AbstractCSP}
    return getindex(problem.vars, findfirst((function(var) return !haskey(assignment, var);
    end),problem.vars));
end

function unordered_domain_values(problem::T, var, assignment::Dict) where {T <: AbstractCSP}
    return choices(problem, var);
end

function choices(problem::T, key) where {T <: AbstractCSP}
    if (!(problem.current_domains === nothing))
        return problem.current_domains[key];
    else
        return problem.domains[key];
    end
end

function no_inference(problem::T, var, value, assignment::Dict, removals::Union{Nothing, AbstractVector}) where {T <: AbstractCSP}
    return true;
end

no_inference (generic function with 1 method)

In [63]:
mutable struct ScheduleCSP <: AbstractCSP
    vars::AbstractVector
    domains::CSPDict
    neighbors::CSPDict
    constraints::Function
    initial::Tuple
    current_domains::Union{Nothing, Dict}
    nassigns::Int64
    function ScheduleCSP(constraints::Function;
        initial::Tuple=(), current_domains::Union{Nothing, Dict}=nothing, nassigns::Int64=0)
        local vars = Course.date
        local domains = Course.available
        local neighbors=Schedule.courses
        return new(vars, domains, neighbors, ScheduleConstraints, initial, current_domains, nassigns)
    end
end

In [64]:
function scheduleConstraints(s::Schedule)
    for c1 in s.courses
        if c1.dates ≠ nothing
            for c2 in s.courses
                if c2.dates ≠ nothing
                    if Day(c1.dates)-Day(c2.dates)<c2.prep_days && c1.dates >= c2.available
                        return false
                    end
                end
            end
        end
        if c1.date ≠ nothing && c1.date ∉ c.available
            return false
        end
    end
    true
end

ScheduleConstraints (generic function with 1 method)

In [65]:
prob = ScheduleCSP

ScheduleCSP

In [67]:
backtracking_search(ScheduleCSP)

LoadError: MethodError: no method matching backtracking_search(::Type{ScheduleCSP})
Closest candidates are:
  backtracking_search(!Matched::T; select_unassigned_variable, order_domain_values, inference) where T<:AbstractCSP at In[62]:1